In [96]:
import pandas as pd
import numpy as np
import os
import gzip
import pickle
import re
import copy
from tqdm import tqdm
import time

In [97]:
hardware_instances = pd.read_csv('hardware_instances.csv') # columns: date,hardware

# # keep only values near the end of 2021
# hardware_instances = hardware_instances[hardware_instances["date"] > "2020-07-01"]
# hardware_instances = hardware_instances[hardware_instances["date"] < "2020-11-01"]

# unique_values = hardware_instances["hardware"].value_counts()
# pairs = []
# for i in range(0, len(unique_values)):
#     # print(unique_values.index[i], unique_values[i])
#     pairs.append((unique_values.index[i], unique_values[i]))

# pairs = sorted(pairs, key=lambda x: x[0], reverse=False)
# for pair in pairs:
#     # print(pair)
#     print(pair[0])

pd.set_option('display.max_rows', None)

# keep only values near the end of 2021
hardware_instances = hardware_instances[hardware_instances["date"] > "2020-07-01"]
hardware_instances = hardware_instances[hardware_instances["date"] < "2020-12-01"]
hardware_instances[["date", "hardware"]].to_csv("hardware_instances_2020_all.csv", index=False)

from hardware_mapping import map_hardware_to_table
hardware_instances = hardware_instances.assign(hardware_mapped = hardware_instances["hardware"].apply(map_hardware_to_table))
# hardware_instances

In [98]:
hardware_instances = pd.read_csv('hardware_instances.csv') # columns: date,hardware
hardware_instances = hardware_instances[hardware_instances["date"] > "2018-01-01"]
hardware_instances = hardware_instances.assign(hardware_mapped = hardware_instances["hardware"].apply(map_hardware_to_table))

# find most common unique values that were mapped to "unknown"
unique_values = hardware_instances[hardware_instances["hardware_mapped"] == "unknown"]["hardware"].value_counts()
unique_values[:100]

hardware
psu                    257
hashboard              132
fan                    109
fans                   101
miner                   92
miners                  83
hashboards              77
sd card                 68
asics                   68
hash boards             59
router                  56
power supply            51
asic                    51
hash board              51
antminer                46
l3+                     36
antminers               33
ethernet cable          31
apw3++                  27
raspberry pi            26
pc                      24
cables                  22
heatsinks               21
asic miners             21
chips                   19
laptop                  19
pcb                     18
s15                     18
asic miner              17
boards                  17
temp sensor             16
cgminer                 16
psus                    16
cable                   16
chip                    16
asic chips              15
data cable         

In [99]:
hardware_instances = pd.read_csv('hardware_instances.csv') # columns: date,hardware
hardware_instances = hardware_instances.assign(hardware_mapped = hardware_instances["hardware"].apply(lambda x: str(x).strip()).apply(map_hardware_to_table))

# print things that are "not found"
unique_values = hardware_instances[hardware_instances["hardware_mapped"] == "not found"]["hardware"].value_counts()
unique_values[:10]

hardware
control board       207
computer            194
controller          141
controller board     72
computers            52
controllers          18
control boards       16
vps                  15
antminer s17         14
4gb ram              13
Name: count, dtype: int64

In [100]:
instances = pd.read_csv('hardware_instances_inc_threads.csv') # columns: date,thread


efficiency = pd.read_csv('hardware_instances_with_efficiency.csv') # columns: date,hardware_name,TH/J
efficiency["ln_efficiency"] = np.log10(efficiency["TH/J"]*1000000000000)
# efficiency = efficiency[efficiency["date"] > "2013-01-01"]
efficiency = efficiency[efficiency["date"] > "2015-07-01"]
efficiency = efficiency[efficiency["ln_efficiency"] < 7.5]

i = 0
for index, row in efficiency.iterrows():
    print(row["date"], row["hardware_name"], row["ln_efficiency"])
    filtered = instances[instances["date"] == row["date"]]
    print(filtered.iloc[0]["thread"])
    print("\n\n\n\n\n\n\n\n\n\n\n\n")
    # if i > 5:
    #     break
    i += 1

2015-10-31 02:51:44 9600gt 5.217483944213907
User:
In the given Bitcoin forum thread, pay close attention to the language used when mentioning hardware pieces. Look for explicit statements indicating ownership or hypothetical discussions.

```thread
Date: 2015-10
Topic: No connection could be made because the target machine actively refused it?
### Original post:
Hello,Whenever I try to mine on Eligius or solo using GUIMiner, I get these errors:[Errno 10061] No connection could be made because the target machine actively refused itIO errors - 2, tolerance 2[Errno 10061] No connection could be made because the target machine actively refused itIO errors - 3, tolerance 2No more backup servers left. Using primary and starting over.These all repeat constantly. I read somewhere that you have to press the "Launch Bitcoin client as server" setting, however this does not work as I do not use the core Bitcoin client. I use MultiBit HD. Am I missing something? Must I use the core client to mine?